Importando bibliotecas necessárias para o desenvolvimento

In [34]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

Criando variáveis e consumindo a API da Nasa

In [35]:
api_key = '2A2nf4ZGr0WgpzKNFF16W9clGhMaP1OTpnc9RgOF'
base_url = 'https://api.nasa.gov/neo/rest/v1/feed'

params = {
    'api_key': api_key
}

response = requests.get(base_url, params=params)
data = response.json()

Visualizando como a API está distribuída


In [36]:
data

{'links': {'next': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2024-11-02&end_date=2024-11-09&detailed=false&api_key=2A2nf4ZGr0WgpzKNFF16W9clGhMaP1OTpnc9RgOF',
  'previous': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2024-10-19&end_date=2024-10-26&detailed=false&api_key=2A2nf4ZGr0WgpzKNFF16W9clGhMaP1OTpnc9RgOF',
  'self': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2024-10-26&end_date=2024-11-02&detailed=false&api_key=2A2nf4ZGr0WgpzKNFF16W9clGhMaP1OTpnc9RgOF'},
 'element_count': 112,
 'near_earth_objects': {'2024-10-30': [{'links': {'self': 'http://api.nasa.gov/neo/rest/v1/neo/2177016?api_key=2A2nf4ZGr0WgpzKNFF16W9clGhMaP1OTpnc9RgOF'},
    'id': '2177016',
    'neo_reference_id': '2177016',
    'name': '177016 (2003 BM47)',
    'nasa_jpl_url': 'https://ssd.jpl.nasa.gov/tools/sbdb_lookup.html#/?sstr=2177016',
    'absolute_magnitude_h': 20.04,
    'estimated_diameter': {'kilometers': {'estimated_diameter_min': 0.2609486033,
      'estimated_diameter_max': 0.5834988155},


Tratando a api e gerando uma lista com os dados que foram selecionados

In [37]:
asteroids = []
for date, asteroids_list in data['near_earth_objects'].items():
    for asteroid in asteroids_list:
        asteroids.append({
            'name': asteroid['name'],
            'close_approach_date': date,
            'velocity_kmh': asteroid['close_approach_data'][0]['relative_velocity']['kilometers_per_hour'],
            'estimated_diameter_min': asteroid['estimated_diameter']['meters']['estimated_diameter_min'],
            'estimated_diameter_max': asteroid['estimated_diameter']['meters']['estimated_diameter_max'],
            "miss_distance_km": asteroid['close_approach_data'][0]['miss_distance']['kilometers'],
            'is_potentially_hazardous': asteroid['is_potentially_hazardous_asteroid']
        })

Criando o DataFrame a partir da lista 'asteroids'

In [38]:
df_asteroids = pd.DataFrame(asteroids)

Visualizando as primeiras linhas

In [39]:
df_asteroids.head()

,name,close_approach_date,velocity_kmh,estimated_diameter_min,estimated_diameter_max,miss_distance_km,is_potentially_hazardous
0,177016 (2003 BM47),2024-10-30,59417.8031569194,260.948603,583.498816,37903968.544348044,False
1,519354 (2011 KR12),2024-10-30,52626.6782736617,135.068780,302.022973,54400987.786813067,False
2,524392 (2002 AU5),2024-10-30,90292.9423154634,728.710415,1629.446024,72807443.364957722,False
3,(2011 UB276),2024-10-30,75513.8578737028,62.886146,140.617696,55115306.411182604,False
4,(2013 TH4),2024-10-30,62275.4391105904,62.309602,139.328506,52496776.740174878,False


Visualizando informações sobre o DataFrame, como o tipo de dado de cada coluna

In [40]:
df_asteroids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   name                      112 non-null    object 
 1   close_approach_date       112 non-null    object 
 2   velocity_kmh              112 non-null    object 
 3   estimated_diameter_min    112 non-null    float64
 4   estimated_diameter_max    112 non-null    float64
 5   miss_distance_km          112 non-null    object 
 6   is_potentially_hazardous  112 non-null    bool   
dtypes: bool(1), float64(2), object(4)
memory usage: 5.5+ KB


Ajustando o tipo de dados para as colunas necessárias

In [47]:
df_asteroids['close_approach_date'] = pd.to_datetime(df_asteroids['close_approach_date'])

df_asteroids['miss_distance_km'] = df_asteroids['miss_distance_km'].astype('float64')

df_asteroids['velocity_kmh'] = df_asteroids['velocity_kmh'].astype('float64')

Descrição estatítisca dos dados das colunas de data e numéricas

In [42]:
df_asteroids.describe()

,close_approach_date,velocity_kmh,estimated_diameter_min,estimated_diameter_max,miss_distance_km
count,112,112.000000,112.000000,112.000000,1.120000e+02
mean,2024-10-29 04:30:00,49095.736424,106.970149,239.192525,3.626349e+07
min,2024-10-26 00:00:00,7176.497894,1.572371,3.515928,2.123893e+05
25%,2024-10-27 18:00:00,27352.641203,15.374709,34.378895,1.434775e+07
50%,2024-10-29 00:00:00,47700.565592,36.270716,81.103786,3.717393e+07
75%,2024-10-31 00:00:00,65112.026966,95.025154,212.482705,5.694685e+07
max,2024-11-02 00:00:00,129730.167342,1301.832102,2910.985075,7.466000e+07
std,NaN,26028.932988,187.784087,419.897983,2.303957e+07
